In [1]:
%load_ext autoreload
%autoreload 2

from wordle import Wordle 


In [2]:
# new way of inputting information 

# Here to input information, put in your five letter guess, 
# followed by the response in the following way: 
#  put ('guess','response'), on the list, where each character of 'response' 
#  indicates the color that the corresponding character of the guess turned: 
#    *  if the Tile turned green,  put 'g'
#    *  if the Tile turned yellow, put 'y'
#    *  if the Tile turned gray,   put 'x' 
# so for example if you guessed 'tears', and the correct word was 'rebut', 
#       the response string would be 'ygxyx'

# You should always start with the guess 'tears' or 'soare' or 'trace' or 'crane', rather than re-running this program 
wordle = Wordle() # create instance of wordle words object

Guesses = [
    ('soare','gxxxx'),  
    ('sight','gyxxx'),
    # ('slick','gygxy'),
#     ('joint','xgggg')
]

wordle.add_guesses(Guesses, reset_info=True) 

In [3]:
valid_words = wordle.get_possible_words()

print(f'Found {len(valid_words)} valid_words')
if len(valid_words) < 50: 
    print(valid_words)
else: 
    print("Too many words to display")

Found 13 valid_words
['skiff', 'skill', 'skimp', 'slick', 'slimy', 'slink', 'sniff', 'spicy', 'spiky', 'spill', 'spiny', 'squib', 'swill']


In [4]:
opt_guesses = wordle.get_best_guesses(use_tqdm=False)
opt_guesses

using hard mode: False
best guess found gave 2.458311329683084 bits of information


[{'guess': 'plank', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plink', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plonk', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'pluck', 'num_classes': 12, 'entropy bits': 2.458311329683084},
 {'guess': 'plunk', 'num_classes': 12, 'entropy bits': 2.458311329683084}]